# 故障排除

有时会出现错误，但我们在这里帮助您！本指南涵盖了一些最常见的问题及其解决方法。然而，这并不是一个涵盖所有 🤗 Transformers 问题的详尽清单。如果您需要更多帮助，可以尝试以下方法：

1. 在 [论坛](https://discuss.huggingface.co/)上寻求帮助。您可以将问题发布到特定的类别中，例如 [初学者](https://discuss.huggingface.co/c/beginners/5) 或 [🤗 Transformers](https://discuss.huggingface.co/c/transformers/9)。确保您的帖子描述清楚并包含可复现的代码，以提高解决问题的可能性！

2. 如果是与库相关的bug，请在 🤗 Transformers 仓库中创建一个 [问题](https://github.com/huggingface/transformers/issues/new/choose)。尽量提供尽可能多的信息来描述bug，以便我们更好地找出问题所在并解决它。

3. 如果您使用的是较旧版本的 🤗 Transformers，请查看 [迁移指南](migration)，因为不同版本之间引入了一些重要的更改。

有关故障排除和获取帮助的更多详细信息，请参阅 Hugging Face 课程的 [第8章](https://huggingface.co/course/chapter8/1?fw=pt)。

## 网络隔离环境

某些云上的GPU实例或内部网络设置可能被防火墙隔离，导致连接错误。当您的脚本尝试下载模型权重或数据集时，下载会挂起，然后超时，并显示以下消息：



在这种情况下，您应该尝试在[离线模式](installation#offline-mode)下运行 🤗 Transformers 以避免连接错误。

## CUDA 内存不足

没有适当的硬件，训练具有数百万参数的大模型可能会很有挑战性。当GPU内存不足时，您可能会遇到以下常见错误：



以下是一些减少内存使用的潜在解决方案：

* 减少 [TrainingArguments](/docs/transformers/v4.46.3/en/main_classes/trainer#transformers.TrainingArguments) 中的 `per_device_train_batch_size` 值。
* 尝试在 [TrainingArguments](/docs/transformers/v4.46.3/en/main_classes/trainer#transformers.TrainingArguments) 中使用 `gradient_accumulation_steps` 来有效增加整体批次大小。

有关节省内存技巧的更多详细信息，请参阅 [性能指南](performance)。

## 无法加载保存的 TensorFlow 模型

TensorFlow 的 [model.save](https://www.tensorflow.org/tutorials/keras/save_and_load#save_the_entire_model) 方法会将整个模型（包括架构、权重和训练配置）保存在一个文件中。但是，当您再次加载模型文件时，可能会遇到错误，因为 🤗 Transformers 可能无法加载模型文件中的所有 TensorFlow 相关对象。为了避免保存和加载 TensorFlow 模型时出现问题，我们建议您：

* 使用 `model.save_weights` 将模型权重保存为 `.h5` 文件扩展名，然后使用 `from_pretrained()` 重新加载模型：


In [ ]:
from transformers import TFPreTrainedModel
from tensorflow import keras

model.save_weights("some_folder/tf_model.h5")
model = TFPreTrainedModel.from_pretrained("some_folder")


* 使用 `~TFPretrainedModel.save_pretrained` 保存模型，并使用 `from_pretrained()` 重新加载模型：


In [ ]:
from transformers import TFPreTrainedModel

model.save_pretrained("path_to/model")
model = TFPreTrainedModel.from_pretrained("path_to/model")


## 导入错误

您可能会遇到的另一个常见错误是 `ImportError`，尤其是对于新发布的模型：



对于这些类型的错误，请确保您安装了最新版本的 🤗 Transformers 以访问最新的模型：


In [ ]:
pip install transformers --upgrade


## CUDA 错误：设备端断言触发

有时您可能会遇到一个通用的 CUDA 错误，提示设备代码中有错误：



您应该首先在 CPU 上运行代码以获取更详细的错误消息。在代码开头添加以下环境变量以切换到 CPU：


In [ ]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = ""


另一种选择是获取更好的 GPU 回溯信息。在代码开头添加以下环境变量以获取指向错误源的回溯：


In [ ]:
import os

os.environ["CUDA_LAUNCH_BLOCKING"] = "1"


## 当填充标记未被屏蔽时输出不正确

在某些情况下，如果 `input_ids` 包含填充标记，输出 `hidden_state` 可能不正确。为了演示这一点，加载一个模型和分词器。您可以访问模型的 `pad_token_id` 来查看其值。对于某些模型，`pad_token_id` 可能为 `None`，但您可以始终手动设置它。


In [ ]:
from transformers import AutoModelForSequenceClassification
import torch

model = AutoModelForSequenceClassification.from_pretrained("google-bert/bert-base-uncased")
model.config.pad_token_id


以下示例展示了未屏蔽填充标记时的输出：


In [ ]:
input_ids = torch.tensor([[7592, 2057, 2097, 2393, 9611, 2115], [7592, 0, 0, 0, 0, 0]])
output = model(input_ids)
print(output.logits)


以下是第二个序列的实际输出：


In [ ]:
input_ids = torch.tensor([[7592]])
output = model(input_ids)
print(output.logits)


大多数情况下，您应该向模型提供 `attention_mask` 以忽略填充标记，以避免这种无声错误。现在，第二个序列的输出与其实际输出匹配：

默认情况下，分词器会根据您使用的特定分词器的默认设置为您创建 `attention_mask`。


In [ ]:
attention_mask = torch.tensor([[1, 1, 1, 1, 1, 1], [1, 0, 0, 0, 0, 0]])
output = model(input_ids, attention_mask=attention_mask)
print(output.logits)


🤗 Transformers 不会在提供填充标记时自动创建 `attention_mask` 以屏蔽填充标记，因为：

* 有些模型没有填充标记。
* 对于某些用途，用户希望模型关注填充标记。

## ValueError: 无法识别此种类的 AutoModel 的配置类 XYZ

通常，我们建议使用 [AutoModel](/docs/transformers/v4.46.3/en/model_doc/auto#transformers.AutoModel) 类来加载预训练模型。这个类可以根据给定检查点的配置自动推断并加载正确的架构。如果您在从检查点加载模型时看到此 `ValueError`，这意味着 Auto 类无法将给定检查点中的配置映射到您尝试加载的模型类型。最常见的原因是检查点不支持给定任务。例如，您会在以下示例中看到此错误，因为 GPT2 没有用于问答的任务模型：


In [ ]:
from transformers import AutoProcessor, AutoModelForQuestionAnswering

processor = AutoProcessor.from_pretrained("openai-community/gpt2-medium")
model = AutoModelForQuestionAnswering.from_pretrained("openai-community/gpt2-medium")